In [7]:
!pip3 install cupy

  Using cached cupy-9.6.0.tar.gz (1.6 MB)
ERROR: Operation cancelled by user


In [4]:
import pandas as pd
import numpy as np
import torch
import json
import itertools


def load_recommender_data(path_in_str):
    json_file = open(path_in_str)
    data = json.load(json_file)
    df = pd.DataFrame.from_dict(data, orient='index')
    df = df[df['error'].isna()]

    # %%
    df['user'] = df.index
    df.reset_index(inplace=True)
    df['user_id'] = df.index

    # %%
    def melt_series(s):
        lengths = s.str.len().values
        flat = [i for i in itertools.chain.from_iterable(s.values.tolist())]
        idx = np.repeat(s.index.values, lengths)
        return pd.Series(flat, idx, name=s.name)


    user_df = melt_series(df.data).to_frame().join(df.drop('data', 1))

    # %%
    user_df['anime_id'] = user_df['data'].apply(lambda x : x['node']['id'])
    user_df['status'] = user_df['data'].apply(lambda x : x['list_status']['status'])
    user_df['score'] = user_df['data'].apply(lambda x : x['list_status']['score'])
    user_df['is_rewatching'] = user_df['data'].apply(lambda x : x['list_status']['is_rewatching'])


    # %%
    user_df = user_df[['user', 'user_id', 'anime_id', 'status', 'score', 'is_rewatching']]
    user_df = user_df.rename({'score': 'user_score'}, axis=1)

    # %%
    path_in_str = '/content/data/anime_list_final_231.json'
    json_file = open(path_in_str)
    data = json.load(json_file)
    anime_df_raw = pd.DataFrame.from_dict(data, orient='index')
    anime_df_raw = anime_df_raw[anime_df_raw['error'] != 'not_found']
    anime_df = anime_df_raw[['id', 'title', 'mean', 'genres', 'statistics', 'num_episodes']]
    anime_df = anime_df.dropna()

    # %%
    anime_df['genres_name'] = anime_df['genres'].apply(lambda x : [a['name'] for a in x])
    anime_df['genres_id'] = anime_df['genres'].apply(lambda x : [a['id'] for a in x])

    # %%
    anime_df['watching'] = anime_df['statistics'].apply(lambda x : x['status']['watching'])
    anime_df['num_list_users'] = anime_df['statistics'].apply(lambda x : x['num_list_users'])
    anime_df['completed'] = anime_df['statistics'].apply(lambda x : x['status']['completed'])
    anime_df['plan_to_watch'] = anime_df['statistics'].apply(lambda x : x['status']['plan_to_watch'])
    anime_df['dropped'] = anime_df['statistics'].apply(lambda x : x['status']['dropped'])
    anime_df['on_hold'] = anime_df['statistics'].apply(lambda x : x['status']['on_hold'])

    # %%
    anime_df.drop(['genres', 'statistics'], axis=1, inplace=True)

    # %%
    anime_df.rename({'id': 'anime_id'}, axis=1, inplace=True)



    # %%
    df_merge_raw = pd.merge(anime_df, user_df, on = 'anime_id')

    # %%
    df_merge = df_merge_raw[['anime_id', 'title', 'genres_name', 'num_episodes', 'mean', 'num_list_users', 'user_id', 'user_score']]

    # %%
    df_merge.rename({'title': 'name', 'genres_name': 'genre', 'num_episodes': 'episodes', 'mean': 'rating_x', 'num_list_users': 'members', 'user_score': 'rating_y'}, axis=1, inplace= True)

    # %%
    df_merge['rating_x'] = df_merge['rating_x'].astype(int)
    df_merge['rating_x'] = df_merge['rating_x'].round()

    return df_merge, anime_df, user_df


def main():
    path_in_str = '/content/user_data1.0.json'
    load_recommender_data(path_in_str)


In [12]:
df_merge, anime_df, user_df =  load_recommender_data('user_data1.0.json')
df_merge.head()
sample = df_merge.sample(n=10000)
sample.shape

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

(10000, 8)

In [13]:
df_merge.dtypes

anime_id    float64
name         object
genre        object
episodes    float64
rating_x      int64
members       int64
user_id       int64
rating_y      int64
dtype: object

In [14]:
df_merge['anime_id'] = df_merge['anime_id'].astype(np.int32)
df_merge['rating_x'] = df_merge['rating_x'].astype(np.int32)
df_merge['user_id'] = df_merge['user_id'].astype(np.int32)
df_merge = df_merge[['anime_id', 'rating_x','user_id']]
df_merge.dtypes

anime_id    int32
rating_x    int32
user_id     int32
dtype: object

In [15]:
df_merge = df_merge.sample(n=5)
df_merge

,anime_id,rating_x,user_id
4128,3297,8,39
43797,37210,6,34
26784,1573,6,40
28075,3655,7,68
23941,46420,7,44


In [16]:
ratings_mat = np.ndarray(
    shape=(np.max(df_merge.anime_id.values), np.max(df_merge.user_id.values)+1),
    dtype=np.uint8)
ratings_mat

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,  58,  64, ...,  64,   0,   0],
       [  0,   0,   0, ...,   0,  64,  81],
       ...,
       [127,   0,   0, ...,   0,  48, 199],
       [131,  67,   7, ...,   7, 127,   0],
       [  0,  32,  93, ...,  83, 130,  67]], dtype=uint8)

In [17]:
ratings_mat[df_merge.anime_id.values-1, df_merge.user_id.values] = df_merge.rating_x.values
ratings_mat

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,  58,  64, ...,  64,   0,   0],
       [  0,   0,   0, ...,   0,  64,  81],
       ...,
       [127,   0,   0, ...,   0,  48, 199],
       [131,  67,   7, ...,   7, 127,   0],
       [  0,  32,  93, ...,  83, 130,  67]], dtype=uint8)

In [18]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T
normalised_mat

array([[-15.46376812, -15.46376812, -15.46376812, ..., -15.46376812,
        -15.46376812, -15.46376812],
       [-26.30434783,  31.69565217,  37.69565217, ...,  37.69565217,
        -26.30434783, -26.30434783],
       [-33.33333333, -33.33333333, -33.33333333, ..., -33.33333333,
         30.66666667,  47.66666667],
       ...,
       [ 54.42028986, -72.57971014, -72.57971014, ..., -72.57971014,
        -24.57971014, 126.42028986],
       [ 60.20289855,  -3.79710145, -63.79710145, ..., -63.79710145,
         56.20289855, -70.79710145],
       [-58.23188406, -26.23188406,  34.76811594, ...,  24.76811594,
         71.76811594,   8.76811594]])

In [19]:
cuda  = torch.device('cuda')
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
t = torch.from_numpy(A).to(device=cuda) # converted to tensor
U, S, Vt = torch.svd(t)
#convert Vt to numpy array
V = Vt.cpu().detach().numpy()

In [21]:
V

array([[-1.04219483e-03, -5.14701179e-03,  3.34457355e-03, ...,
        -7.80519301e-05, -3.39384317e-03, -7.09223406e-02],
       [-6.42086092e-03,  4.37885329e-03,  2.32905084e-05, ...,
        -5.88521365e-04, -1.69649250e-03, -4.31833183e-01],
       [ 9.51498218e-03,  2.67743577e-03, -5.62241410e-03, ...,
        -1.07577208e-03,  1.61065594e-03, -1.86363493e-01],
       ...,
       [ 8.01653178e-03, -4.06049904e-03, -1.16704843e-02, ...,
         3.08556749e-03,  2.44187318e-03,  1.09683676e-02],
       [-7.60380817e-03, -3.37293162e-03, -5.20888016e-03, ...,
         1.29549396e-03, -1.45965998e-03,  5.13638586e-03],
       [ 4.52890799e-03,  4.42942849e-03,  3.83361538e-03, ...,
         9.64266386e-03,  3.07187952e-03, -4.07923035e-03]])

In [54]:
def top_cosine_similarity(data, anime_id, top_n=10):
    index = anime_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

In [55]:
def print_similar_movies(anime_df, anime_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    anime_df[anime_df.anime_id == anime_id].title))
    for id in top_indexes + 1:
        print(anime_df[anime_df.anime_id == id].title.values)

In [56]:
k = 50
anime_id = 6 # (getting an id from movies.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, anime_id, top_n)

#Printing the top N similar movies
print_similar_movies(anime_df, anime_id, indexes)

Recommendations for 6    Trigun
Name: title, dtype: object: 

['Trigun']
['Sunabouzu']
['Akira']
[]
['Rozen Maiden: Träumend']
['Blue Gender']
[]
['Yakitate!! Japan']
[]
['Ai Yori Aoshi']
